In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [5]:
# Import the tweets dataset, this is a cut down version of the full tweet 
# dataset per the operations performed in the file "eda-rea-v-liv-2018"
en_tweets_df = pd.read_csv('en_tweets_df.csv', lineterminator='\n')

In [11]:
en_tweets_df.head()

,Unnamed: 0,id,text,lang,retweeted_status,created_at,user,tweet_text,is_retweet,created_at_hour_minute
0,0,1.000366e+18,MATCH-DAY\n\nReal Madrid vs Liverpool\n\n#UCLF...,en,NaN,2018-05-26 13:18:30+00:00,"{'id': 2846595478, 'id_str': '2846595478', 'na...",MATCH-DAY\n\nReal Madrid vs Liverpool\n\n#UCLF...,False,2018-05-26 13:18:00+00:00
1,4,1.000366e+18,RT @ECG_Unofficial: We will like to categorica...,en,{'created_at': 'Sat May 26 11:14:44 +0000 2018...,2018-05-26 13:18:31+00:00,"{'id': 902735000445095938, 'id_str': '90273500...",RT @ECG_Unofficial: We will like to categorica...,True,2018-05-26 13:18:00+00:00
2,5,1.000366e+18,Real Madrid.... LETS GO!!!!!!!!!!!!!!!!!!!!!!!...,en,NaN,2018-05-26 13:18:31+00:00,"{'id': 240672622, 'id_str': '240672622', 'name...",Real Madrid.... LETS GO!!!!!!!!!!!!!!!!!!!!!!!...,False,2018-05-26 13:18:00+00:00
3,7,1.000366e+18,RT @YNWA_Claire: Please? 🙏🏼\n\n#UCLFinal https...,en,{'created_at': 'Sat May 26 13:13:45 +0000 2018...,2018-05-26 13:18:31+00:00,"{'id': 277019564, 'id_str': '277019564', 'name...",RT @YNWA_Claire: Please? 🙏🏼\n\n#UCLFinal https...,True,2018-05-26 13:18:00+00:00
4,8,1.000366e+18,RT @ChampionsLeague: Two European giants go he...,en,{'created_at': 'Sat May 26 13:00:01 +0000 2018...,2018-05-26 13:18:31+00:00,"{'id': 957644286350315521, 'id_str': '95764428...",RT @ChampionsLeague: Two European giants go he...,True,2018-05-26 13:18:00+00:00


In [13]:
# Import NLTK libraries
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /Users/jamie/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [47]:
# word_tokenize
en_tweets_df['tokenized_words'] = en_tweets_df.apply(lambda row: word_tokenize(row['tweet_text']), axis=1)


In [49]:
from collections import Counter

# word_tokenized = word_tokenize()
bow = Counter(en_tweets_df['tokenized_words'][0])
print(bow.most_common())

[(':', 3), ('#', 2), ('https', 2), ('MATCH-DAY', 1), ('Real', 1), ('Madrid', 1), ('vs', 1), ('Liverpool', 1), ('UCLFinal', 1), ('LFC', 1), ('Free', 1), ('Live', 1), ('Stream', 1), ('HD', 1), ('Here', 1), ('//t.co/PHAepWsA6o', 1), ('//t.co/T6mWNz14lb', 1)]


In [50]:
bow

Counter({'MATCH-DAY': 1,
         'Real': 1,
         'Madrid': 1,
         'vs': 1,
         'Liverpool': 1,
         '#': 2,
         'UCLFinal': 1,
         'LFC': 1,
         'Free': 1,
         'Live': 1,
         'Stream': 1,
         'HD': 1,
         'Here': 1,
         ':': 3,
         'https': 2,
         '//t.co/PHAepWsA6o': 1,
         '//t.co/T6mWNz14lb': 1})